# LinguaDex Backend Implementation with Express and Redis

This notebook demonstrates how to replace the static JS backend with Express and Redis for the LinguaDex application.

## Set Up Express Server

First, we need to install the required packages and initialize an Express application.

In [ ]:
// Install required packages
// Run these in your terminal:
// npm install express cors body-parser dotenv

// Initialize Express application
const express = require('express');
const cors = require('cors');
const bodyParser = require('body-parser');
require('dotenv').config();

const app = express();
const port = process.env.PORT || 3000;

// Set up middleware
app.use(cors());
app.use(bodyParser.json());
app.use(bodyParser.urlencoded({ extended: true }));

// Basic route for testing
app.get('/', (req, res) => {
  res.send('LinguaDex API is running!');
});

// Start the server
app.listen(port, () => {
  console.log(`Server is running on port ${port}`);
});

## Install and Configure Redis

Next, we'll install Redis and set up a Redis client to connect to the server.

In [ ]:
// Install Redis
// Run these in your terminal:
// npm install redis

const redis = require('redis');

// Redis client configuration
const redisClient = redis.createClient({
  url: process.env.REDIS_URL || 'redis://localhost:6379'
});

// Connect to Redis
(async () => {
  try {
    await redisClient.connect();
    console.log('Connected to Redis successfully');
  } catch (error) {
    console.error('Redis connection error:', error);
  }
})();

// Handle Redis errors
redisClient.on('error', (err) => {
  console.error('Redis error:', err);
});

## Replace Static JS with API Endpoints

Now, we'll create Express routes to handle CRUD operations for the LinguaDex application, using Redis for data storage.

In [ ]:
// Data Models
// Let's define some helper functions for handling vocabulary data in Redis

// Vocabulary item structure
// {
//   id: string,
//   term: string,
//   definition: string,
//   language: string,
//   tags: string[],
//   examples: string[],
//   dateAdded: Date,
//   lastReviewed: Date,
//   proficiencyLevel: number (1-5)
// }

// Helper functions for Redis operations
const VOCAB_KEY_PREFIX = 'vocab:';
const LANGUAGE_SET_KEY = 'languages';

// Add a new vocabulary item
async function addVocabularyItem(item) {
  const id = item.id || Date.now().toString();
  item.id = id;
  item.dateAdded = item.dateAdded || new Date().toISOString();
  
  await redisClient.hSet(
    `${VOCAB_KEY_PREFIX}${id}`, 
    Object.entries(item).reduce((obj, [key, value]) => {
      obj[key] = typeof value === 'object' ? JSON.stringify(value) : value;
      return obj;
    }, {})
  );
  
  // Add to language set
  await redisClient.sAdd(LANGUAGE_SET_KEY, item.language);
  // Add to language-specific list
  await redisClient.sAdd(`language:${item.language}`, id);
  
  return id;
}

// Get a vocabulary item by ID
async function getVocabularyItem(id) {
  const item = await redisClient.hGetAll(`${VOCAB_KEY_PREFIX}${id}`);
  if (Object.keys(item).length === 0) return null;
  
  // Parse JSON strings back to arrays
  if (item.tags) item.tags = JSON.parse(item.tags);
  if (item.examples) item.examples = JSON.parse(item.examples);
  
  return item;
}

// Update a vocabulary item
async function updateVocabularyItem(id, updates) {
  const item = await getVocabularyItem(id);
  if (!item) return null;
  
  // Process updates
  const processedUpdates = {};
  for (const [key, value] of Object.entries(updates)) {
    processedUpdates[key] = typeof value === 'object' ? JSON.stringify(value) : value;
  }
  
  await redisClient.hSet(`${VOCAB_KEY_PREFIX}${id}`, processedUpdates);
  
  // If language is updated, update sets
  if (updates.language && updates.language !== item.language) {
    await redisClient.sRem(`language:${item.language}`, id);
    await redisClient.sAdd(`language:${updates.language}`, id);
    await redisClient.sAdd(LANGUAGE_SET_KEY, updates.language);
  }
  
  return id;
}

// Delete a vocabulary item
async function deleteVocabularyItem(id) {
  const item = await getVocabularyItem(id);
  if (!item) return false;
  
  await redisClient.del(`${VOCAB_KEY_PREFIX}${id}`);
  await redisClient.sRem(`language:${item.language}`, id);
  
  return true;
}

// Get all vocabulary items for a language
async function getVocabularyByLanguage(language) {
  const ids = await redisClient.sMembers(`language:${language}`);
  const items = [];
  
  for (const id of ids) {
    const item = await getVocabularyItem(id);
    if (item) items.push(item);
  }
  
  return items;
}

// Get all available languages
async function getAllLanguages() {
  return await redisClient.sMembers(LANGUAGE_SET_KEY);
}

In [ ]:
// API Routes

// Get all languages
app.get('/api/languages', async (req, res) => {
  try {
    const languages = await getAllLanguages();
    res.json(languages);
  } catch (error) {
    console.error('Error getting languages:', error);
    res.status(500).json({ error: 'Failed to retrieve languages' });
  }
});

// Get all vocabulary items for a language
app.get('/api/vocabulary/:language', async (req, res) => {
  try {
    const { language } = req.params;
    const items = await getVocabularyByLanguage(language);
    res.json(items);
  } catch (error) {
    console.error(`Error getting vocabulary for ${req.params.language}:`, error);
    res.status(500).json({ error: 'Failed to retrieve vocabulary items' });
  }
});

// Get a specific vocabulary item
app.get('/api/vocabulary/item/:id', async (req, res) => {
  try {
    const { id } = req.params;
    const item = await getVocabularyItem(id);
    
    if (!item) {
      return res.status(404).json({ error: 'Vocabulary item not found' });
    }
    
    res.json(item);
  } catch (error) {
    console.error(`Error getting vocabulary item ${req.params.id}:`, error);
    res.status(500).json({ error: 'Failed to retrieve vocabulary item' });
  }
});

// Add a new vocabulary item
app.post('/api/vocabulary', async (req, res) => {
  try {
    const newItem = req.body;
    // Validation
    if (!newItem.term || !newItem.definition || !newItem.language) {
      return res.status(400).json({ error: 'Term, definition, and language are required' });
    }
    
    const id = await addVocabularyItem(newItem);
    res.status(201).json({ id, ...newItem });
  } catch (error) {
    console.error('Error adding vocabulary item:', error);
    res.status(500).json({ error: 'Failed to add vocabulary item' });
  }
});

// Update a vocabulary item
app.put('/api/vocabulary/:id', async (req, res) => {
  try {
    const { id } = req.params;
    const updates = req.body;
    
    const updated = await updateVocabularyItem(id, updates);
    if (!updated) {
      return res.status(404).json({ error: 'Vocabulary item not found' });
    }
    
    const updatedItem = await getVocabularyItem(id);
    res.json(updatedItem);
  } catch (error) {
    console.error(`Error updating vocabulary item ${req.params.id}:`, error);
    res.status(500).json({ error: 'Failed to update vocabulary item' });
  }
});

// Delete a vocabulary item
app.delete('/api/vocabulary/:id', async (req, res) => {
  try {
    const { id } = req.params;
    const deleted = await deleteVocabularyItem(id);
    
    if (!deleted) {
      return res.status(404).json({ error: 'Vocabulary item not found' });
    }
    
    res.status(204).send();
  } catch (error) {
    console.error(`Error deleting vocabulary item ${req.params.id}:`, error);
    res.status(500).json({ error: 'Failed to delete vocabulary item' });
  }
});

## Test API Endpoints

Now let's test our API endpoints using curl commands. You can run these in your terminal to verify that the API is working correctly.

In [ ]:
# Test the server is running
curl http://localhost:3000

# Add a new vocabulary item
curl -X POST http://localhost:3000/api/vocabulary \
  -H "Content-Type: application/json" \
  -d '{
    "term": "Bonjour",
    "definition": "Hello",
    "language": "French",
    "tags": ["greeting", "basic"],
    "examples": ["Bonjour, comment ça va?"],
    "proficiencyLevel": 1
  }'

# Get all languages
curl http://localhost:3000/api/languages

# Get all vocabulary items for a language
curl http://localhost:3000/api/vocabulary/French

# Update a vocabulary item (replace {id} with actual id)
curl -X PUT http://localhost:3000/api/vocabulary/{id} \
  -H "Content-Type: application/json" \
  -d '{
    "proficiencyLevel": 2,
    "examples": ["Bonjour, comment ça va?", "Bonjour tout le monde!"]
  }'

# Delete a vocabulary item (replace {id} with actual id)
curl -X DELETE http://localhost:3000/api/vocabulary/{id}

## Integration with the Frontend

To integrate this backend with your frontend, you'll need to update your frontend code to use these API endpoints instead of the static JavaScript implementation.

Here's a basic example of how to use these endpoints from the frontend:

In [ ]:
// Example frontend code using fetch API

// Get all languages
async function getLanguages() {
  try {
    const response = await fetch('http://localhost:3000/api/languages');
    const languages = await response.json();
    return languages;
  } catch (error) {
    console.error('Error fetching languages:', error);
    return [];
  }
}

// Get vocabulary items for a language
async function getVocabularyItems(language) {
  try {
    const response = await fetch(`http://localhost:3000/api/vocabulary/${language}`);
    const items = await response.json();
    return items;
  } catch (error) {
    console.error(`Error fetching vocabulary for ${language}:`, error);
    return [];
  }
}

// Add a new vocabulary item
async function addVocabularyItem(item) {
  try {
    const response = await fetch('http://localhost:3000/api/vocabulary', {
      method: 'POST',
      headers: {
        'Content-Type': 'application/json',
      },
      body: JSON.stringify(item),
    });
    const newItem = await response.json();
    return newItem;
  } catch (error) {
    console.error('Error adding vocabulary item:', error);
    throw error;
  }
}

// Update a vocabulary item
async function updateVocabularyItem(id, updates) {
  try {
    const response = await fetch(`http://localhost:3000/api/vocabulary/${id}`, {
      method: 'PUT',
      headers: {
        'Content-Type': 'application/json',
      },
      body: JSON.stringify(updates),
    });
    const updatedItem = await response.json();
    return updatedItem;
  } catch (error) {
    console.error(`Error updating vocabulary item ${id}:`, error);
    throw error;
  }
}

// Delete a vocabulary item
async function deleteVocabularyItem(id) {
  try {
    await fetch(`http://localhost:3000/api/vocabulary/${id}`, {
      method: 'DELETE',
    });
    return true;
  } catch (error) {
    console.error(`Error deleting vocabulary item ${id}:`, error);
    throw error;
  }
}

## Conclusion

In this notebook, we've successfully:

1. Set up an Express server with necessary middleware
2. Configured Redis for data persistence
3. Created API endpoints to replace the static JavaScript backend
4. Provided testing commands for each endpoint
5. Shown how to integrate with the frontend

This implementation provides a more robust backend solution for the LinguaDex application with proper data persistence using Redis and a RESTful API with Express.

### Next Steps

1. Add user authentication and authorization
2. Implement data validation with a schema validation library like Joi or Yup
3. Add search functionality for vocabulary items
4. Implement advanced features like spaced repetition for learning
5. Consider containerizing the application with Docker for easier deployment